# Biol 359A  | Principal Component Analysis
### Spring 2024, Week 9
<hr>
Learning Objectives:

  - Understand motivations of dimension reduction
  - Discuss applications towards data visualization and data exxploration
  - Interpret the results of a principal component analysis


In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr24/week9_principalcomponentanalysis.git
!mkdir ./data
!cp week9_principalcomponentanalysis/data/* ./data
!pip install palmerpenguins
!pip install umap-learn
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt

from palmerpenguins import load_penguins

from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (8,8)
COLORS= ["#008080","#CA562C"]

sns.set(font_scale=1.5, rc={'figure.figsize':FIG_SIZE}) 
sns.set_style()
plt.rc("axes.spines", top=False, right=False)


## First we're going to generate a toy dataset

Notice the direction where the most variance occurs. In order to solve for these directions, we calculate the eigenvectors of the covariance matrix and plot it over the original data.

This process is distinct from linear regression. In linear regression we are optimizing a response variable. In PCA there is no labelled data and we are optimizing across all features.

In [ ]:
def example_pca():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    plt.scatter(X[:, 0], X[:, 1])
    plt.axis('equal')
    plt.show()
    pca = PCA(n_components=2)
    pca.fit(X)

    # plot data
    plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector * 3 * np.sqrt(length)
        draw_vector(pca.mean_, pca.mean_ + v)
    plt.axis('equal');
    plt.show()

def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    color='k',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

example_pca()

DISCUSSION QUESTION:
- What would these vectors look like if we did not standardize the data?

ASSIGNMENT QUESTIONS:
- Please answer question 10 in the assignment.

### Coordinate System
Now that we have the eigenvectors, we can do two things with them.
First, we're going to look at using them as a new coordinate system.

In [ ]:
def coordinatesystem_example():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    pca = PCA(n_components=2, whiten=True)
    pca.fit(X)

    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.15)

    # plot data
    ax[0].scatter(X[:, 0], X[:, 1], alpha=0.2)
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector * 3 * np.sqrt(length)
        draw_vector(pca.mean_, pca.mean_ + v, ax=ax[0])
    ax[0].axis('equal');
    ax[0].set(xlabel='x1', ylabel='x2', title='Raw Data')

    # plot principal components
    X_pca = pca.transform(X)
    ax[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.2)
    draw_vector([0, 0], [0, 3], ax=ax[1])
    draw_vector([0, 0], [3, 0], ax=ax[1])
    ax[1].axis('equal')
    ax[1].set(xlabel='Principal component 1', ylabel='Principal component 2',
              title='Transformed Data',
              xlim=(-5, 5), ylim=(-3, 3.1))
coordinatesystem_example()

DISCUSSION QUESTION:
- Discuss the spread of the raw data in 2D. How does this compare to the spread of the data in the new coordinate system?

ASSIGNMENT QUESTION:
- Please answer question 11 in the assignment.

### Projection
The other option we have is to simply use one principal component, and represent our data with one vector rather than two. What does this accomplish? 

In [ ]:
def projection_example():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    pca = PCA(n_components=2, whiten=True)
    pca.fit(X)

    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.15)

    # plot data
    ax[0].scatter(X[:, 0], X[:, 1], alpha=0.5)
    ax[0].axis('equal');
    ax[0].set(xlabel='x', ylabel='y', title='input')

    # plot principal components
    pca = PCA(n_components=1, whiten=True)
    pca.fit(X)
    X_pca = pca.transform(X)
    X_new = pca.inverse_transform(X_pca)
    ax[1].scatter(X[:, 0], X[:, 1], alpha=0.2)
    ax[1].scatter(X_new[:, 0], X_new[:, 1], color="blue", alpha=0.8)
    ax[1].axis('equal')
    ax[1].set(xlabel='x', ylabel='y',
              title='projected data (First PC)')
    
projection_example()

In [ ]:
def PerformPCA(X, n_dimensions=10, plot=True, include_variance=True, fig_width = 10):
    """
    Uses sklearn PCA tool to perform PCA.
    Input:
    X: Pandas DataFrame or NumPy Array of features
    n_dimensions: Number of principal components to fit
    plot: Whether to plot the scree plot
    include_variance: Whether to include variance explained in the column names

    Output:
    X_pca: Pandas dataframe with column titles of PC1,...,PCn including variance explained in labels (optional)
    column_names: List of column names created
    """
    # Standardize the data
    X_standardized = StandardScaler().fit_transform(X)
    
    # Create PCA object and fit to standardized data
    pca = PCA(n_components=n_dimensions)
    pca.fit(X_standardized)
    
    # Transform data into principal components
    X_pca_array = pca.transform(X_standardized)
    
    # Conditionally create column names incorporating the variance explained
    if include_variance:
        column_names = [
            'PC{} ({:.1f}%)'.format(i + 1, var_exp * 100) 
            for i, var_exp in enumerate(pca.explained_variance_ratio_)
        ]
    else:
        column_names = ['PC{}'.format(i + 1) for i in range(n_dimensions)]
    
    # Create DataFrame with new column names
    X_pca = pd.DataFrame(X_pca_array, columns=column_names)
    
    # Optionally plot the cumulative variance explained in a scree plot
    if plot:
        plt.figure(figsize=(fig_width, 6))
        plt.plot(column_names, np.cumsum(pca.explained_variance_ratio_), 'o--')
        plt.title('Scree Plot of Explained Variance', fontsize=14)
        plt.xlabel('Principal Components', fontsize=12)
        plt.ylabel('Cumulative Variance Explained', fontsize=12)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    return X_pca, column_names  # Correctly return both the DataFrame and the column names

### Now we are going to explore a couple datasets that we've looked at before with PCA
We **could** plot all of our data against each other, and that usually makes sense for small datasets.

In [ ]:
penguins=load_penguins()
penguins.dropna(inplace=True)
features=["bill_length_mm", "bill_depth_mm", "flipper_length_mm","body_mass_g"]
label='species'
_ = sns.pairplot(penguins, vars=features, corner=True, hue="species", markers=["o", "s", "D"])

Alternatively, we can determine the principal components of the data and project our data onto the first two principal components.

In [ ]:
penguins_X = penguins[features]
penguins_y = penguins[label]
penguins_PCA_df, column_names = PerformPCA(penguins_X, n_dimensions=4, include_variance=False)
penguins_PCA_df = penguins_PCA_df.join(penguins_y)

# Plot using seaborn with updated axis labels
sns.scatterplot(
    x=column_names[0],  # Use the column name for PC1
    y=column_names[1],  # Use the column name for PC2
    hue='species',
    style='species',
    markers=["o", "s", "D"],
    data=penguins_PCA_df
)
plt.title('PCA of Penguin Dataset')
plt.show()

ASSIGNMENT QUESTION:
- Please answer questions 12 - 15 in the assignment.

We can use box plots to visualize the distribution of different penguin species along the first two principal components by projecting the dataset onto each principal component and plotting the results.

In [ ]:
_ = sns.boxplot(x="species", y="PC1", hue="species",
                 data=penguins_PCA_df)

In [ ]:
_ = sns.boxplot(x="species", y="PC2", hue="species",
                 data=penguins_PCA_df)

Lets see how this compares to a tSNE plot and a UMAP plot.

In [ ]:
from sklearn.manifold import TSNE

def PerformTSNE(X, n_dimensions=2, perplexity=30, plot=True, labels=None):
    """
    Uses sklearn TSNE tool to perform t-SNE
    input:
    X: Pandas DataFrame or Numpy Array of features
    n_dimensions: Number of dimensions to reduce to
    perplexity: Perplexity parameter for t-SNE
    labels: Optional Pandas Series or Numpy Array of labels for plotting

    output:
    X_tsne: Pandas DataFrame with column titles of t-SNE1, ..., t-SNEn
    """
    tsne = TSNE(n_components=n_dimensions, perplexity=perplexity, random_state=42)
    X_tsne_array = tsne.fit_transform(X)
    column_names = ['t-SNE{}'.format(i+1) for i in range(n_dimensions)]
    X_tsne = pd.DataFrame(X_tsne_array, columns=column_names)

    if plot:
        if labels is not None:
            sns.scatterplot(x='t-SNE1', y='t-SNE2', hue=labels, data=X_tsne)
            plt.title('t-SNE Scatter Plot', fontsize=14)
            plt.show()

    return X_tsne



In [ ]:
import umap

def PerformUMAP(X, n_dimensions=2, n_neighbors=15, min_dist=0.1, plot=True, labels=None):
    """
    Uses umap-learn tool to perform UMAP
    input:
    X: Pandas DataFrame or Numpy Array of features
    n_dimensions: Number of dimensions to reduce to
    n_neighbors: Number of neighbors parameter for UMAP
    min_dist: Minimum distance parameter for UMAP
    labels: Optional Pandas Series or Numpy Array of labels for plotting

    output:
    X_umap: Pandas DataFrame with column titles of UMAP1, ..., UMAPn
    """
    reducer = umap.UMAP(n_components=n_dimensions, n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
    X_umap_array = reducer.fit_transform(X)
    column_names = ['UMAP{}'.format(i+1) for i in range(n_dimensions)]
    X_umap = pd.DataFrame(X_umap_array, columns=column_names)

    if plot:
        if labels is not None:
            sns.scatterplot(x='UMAP1', y='UMAP2', hue=labels, data=X_umap)
            plt.title('UMAP Scatter Plot', fontsize=14)
            plt.show()

    return X_umap


### Summary of t-SNE (t-Distributed Stochastic Neighbor Embedding)

[t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) is a powerful non-linear technique for dimensionality reduction for visualizing high-dimensional datasets. It works by converting similarities between data points to [joint probabilities](https://en.wikipedia.org/wiki/Joint_probability_distribution) and trying to minimize the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between the joint probabilities of the low-dimensional embedding and the high-dimensional data. This approach helps t-SNE excel at creating visually compelling clusterings and layouts where similar data points are modeled by nearby points and dissimilar data points are modeled by distant points. Its effectiveness, however, can depend significantly on the correct setting of its perplexity parameter, which roughly determines how to balance attention between local and global aspects of your data.

In [ ]:
# t-SNE
penguins_TSNE_df = PerformTSNE(penguins_X, n_dimensions=2)
penguins_TSNE_df = penguins_TSNE_df.join(penguins_y)

sns.scatterplot(x='t-SNE1', y='t-SNE2', hue='species', data=penguins_TSNE_df, style='species', markers=["o", "s", "D"])
plt.show()

sns.boxplot(x="species", y="t-SNE1", hue="species", data=penguins_TSNE_df)
plt.show()

sns.boxplot(x="species", y="t-SNE2", hue="species", data=penguins_TSNE_df)
plt.show()


### Summary of UMAP (Uniform Manifold Approximation and Projection)

[UMAP](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction#Uniform_manifold_approximation_and_projection) is another non-linear technique for dimensionality reduction. It operates by constructing a high-dimensional graph representing the manifold structure of the data and then optimally laying out this graph in a lower-dimensional space. This process involves approximating the geometric structure of the data through local manifold approximations and fuzzy set theory. UMAP is particularly effective at preserving both local and global data structures, making it suitable for a broad range of applications. Like t-SNE, the performance of UMAP can be influenced by parameters, notably the number of neighbors and the minimum distance between points in the low-dimensional space. These parameters help control how UMAP balances the emphasis on local versus global features of the data.

In [ ]:
# UMAP
penguins_UMAP_df = PerformUMAP(penguins_X, n_dimensions=2)
penguins_UMAP_df = penguins_UMAP_df.join(penguins_y)

sns.scatterplot(x='UMAP1', y='UMAP2', hue='species', data=penguins_UMAP_df, style='species', markers=["o", "s", "D"])
plt.show()

sns.boxplot(x="species", y="UMAP1", hue="species", data=penguins_UMAP_df)
plt.show()

sns.boxplot(x="species", y="UMAP2", hue="species", data=penguins_UMAP_df)
plt.show()

DISCUSSION QUESTIONS:
- What are key difference in the transformed data across these three approaches?
- What are similarities in the transformed data across these three approaches?

---

## We are going to look at our trusty breast cancer dataset

Remember, we could build fairly effective classification models with this dataset.

In [ ]:
cancer_raw = load_breast_cancer()
print(cancer_raw.DESCR)
tumor_features = pd.DataFrame(cancer_raw.data, columns=cancer_raw.feature_names)
tumor = pd.DataFrame(cancer_raw.target, columns=['tumor'])
tumor_nominal = tumor.replace({'tumor': {0: 'benign', 1: 'malignant'}})
cancer_df = pd.concat([tumor_features, tumor_nominal], axis=1)
tumor_features.describe()

In [ ]:
# Perform PCA on the tumor features
PCA_df, column_names = PerformPCA(tumor_features, n_dimensions=30, include_variance=True, fig_width=14) 
# PCA_df, column_names = PerformPCA(tumor_features, n_dimensions=30, include_variance=True, plot=False)  # Unpack the tuple
PCA_df = PCA_df.join(tumor_nominal) 

# Plot the results
sns.scatterplot(x=column_names[0], y=column_names[1], style='tumor', hue='tumor', data=PCA_df, markers=True)
plt.title('PCA of Breast Cancer Dataset')
plt.show()

ASSIGNMENT QUESTION
- Please answer question 16 in the assignment

But if I plot different PCs...

In [ ]:
sns.scatterplot(x=column_names[3], y=column_names[4],style='tumor',hue='tumor',data=PCA_df,markers=True)
plt.show()

ASSIGNMENT QUESTION:
- Please answer question 17 in the assignment

Lets compare to tSNE and UMAP again.

In [ ]:
# t-SNE
tumor_TSNE_df = PerformTSNE(tumor_features, n_dimensions=2)
tumor_TSNE_df = tumor_TSNE_df.join(tumor_nominal)

sns.scatterplot(x='t-SNE1', y='t-SNE2', hue='tumor', data=tumor_TSNE_df, style='tumor', markers=True)
plt.show()

sns.boxplot(x="tumor", y="t-SNE1", hue="tumor", data=tumor_TSNE_df)
plt.show()

sns.boxplot(x="tumor", y="t-SNE2", hue="tumor", data=tumor_TSNE_df)
plt.show()

In [ ]:
# UMAP
tumor_UMAP_df = PerformUMAP(tumor_features, n_dimensions=2)
tumor_UMAP_df = tumor_UMAP_df.join(tumor_nominal)

sns.scatterplot(x='UMAP1', y='UMAP2', hue='tumor', data=tumor_UMAP_df, style='tumor', markers=True)
plt.show()

sns.boxplot(x="tumor", y="UMAP1", hue="tumor", data=tumor_UMAP_df)
plt.show()

sns.boxplot(x="tumor", y="UMAP2", hue="tumor", data=tumor_UMAP_df)
plt.show()

ASSIGNMENT QUESTION:
- Please answer question 18 in the assignment